In [1]:
from ai import AI

myai = AI()

c:\Users\Waadl\Documents\aadildev\projects\etsy-automation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename = "Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf"
sheet_music_name = myai.create_sheet_name(filename)
sheet_music_name

'Avengers Theme - Infinity War Arrangement (Piano Sheet Music PDF)'

In [3]:
myai.create_slug(sheet_music_name)

{'slug': 'avengers-theme-infinity-war-arrangement-sheet-music'}


'avengers-theme-infinity-war-arrangement-sheet-music'

# Test

# Create thumbnail

In [ ]:
from PIL import Image
import os
import fitz
import re


def scale_image(image: Image.Image, scale_factor: float) -> Image.Image:
    """
    Scales up a PIL image by a given factor, maintaining the aspect ratio.

    Parameters:
    - image (PIL.Image.Image): The input image to scale.
    - scale_factor (float): The factor by which to scale the image.

    Returns:
    - PIL.Image.Image: The scaled-up image.
    """
    # Calculate new dimensions while maintaining aspect ratio
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)
    
    # Resize the image with the new dimensions
    scaled_image = image.resize((new_width, new_height), Image.LANCZOS)
    return scaled_image
    

def create_thumbnail_and_cover(filename: str):
    THUMBNAIL_SIZE = [600, 600]
    THUMBNAIL_SHEET_MUSIC_HEIGHT = 546

    if not os.path.exists("thumbnails"):
        os.makedirs("thumbnails")

    if not os.path.exists("tempimages"):
        os.makedirs("tempimages")
    
    if not os.path.exists("covers"):
        os.makedirs("covers")

    # 1. extract the first page of the pdf as an image
    filepath = f"sheetmusic/{filename}"
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File {filename} not found")
    pdf = fitz.open(filepath)
    pdf_first_page = next(pdf.pages())
    pix = pdf_first_page.get_pixmap()

    img_name = re.split(r"\.", filename)[0]

    pdf_image_path = f"tempimages/{img_name}.jpg"
    pix.save(pdf_image_path)

    """ 
    create thumbnail
    """

    # 2. create a thumbnail image with the sheet music image
    background_image = Image.open("thumbnail-background.jpg")
    pdf_image = Image.open(pdf_image_path)

    # max height of 536, maintains aspect ratio
    pdf_image.thumbnail((THUMBNAIL_SIZE[0], THUMBNAIL_SHEET_MUSIC_HEIGHT))
    pdf_image.save(pdf_image_path)

    background_image.paste(pdf_image, (106, 34))
    background_image.save(f"thumbnails/{img_name}.jpg")

    """ 
    create cover
    """

    background_image = Image.open("cover-background.jpg")
    pdf_image_2 = Image.open(pdf_image_path)

    pdf_image_2 = scale_image(pdf_image_2, 1.2)

    background_image.paste(pdf_image_2, (46, 34))
    background_image.save(f"covers/{img_name}.jpg")

    os.remove(pdf_image_path)
    return f"thumbnails/{img_name}.jpg", f"covers/{img_name}.jpg"
    

In [47]:
create_thumbnail_and_cover("Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf")

('thumbnails/Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.jpg',
 'covers/Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.jpg')